In [2]:
from mysql import connector
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)
print("설치 완료")

2.2.3
2.2.5
설치 완료


## DB 연결 코드
- 여기서부터 시작

In [ ]:
import os
from mysql import connector

PASSWORD = 'kmh1234'

try:
    with connector.connect(
        host = "localhost", # 데이터베이스 호스트 주소     
        user = "root",      # 사용자 이름
        password = PASSWORD # 비밀번호
    ) as database: 
        print(f"Database object: {database}")  # 연결된 데이터베이스 객체 출력
except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x000002745FC7EC10>


In [28]:
from dotenv import load_dotenv
import os
import mysql.connector
from mysql.connector import pooling

# .env 파일 로드
load_dotenv()

# 데이터베이스 연결 설정
db_config = {
    'host': os.getenv('MYSQL_HOST'),
    'port': int(os.getenv('MYSQL_PORT')),
    'user': os.getenv('MYSQL_USER'),
    'password': os.getenv('MYSQL_PASSWORD'),
    'database': os.getenv('MYSQL_DATABASE'),
    'charset': os.getenv('MYSQL_CHARSET')
}

# 커넥션 풀 설정
pool_config = {
    'pool_name': os.getenv('MYSQL_POOL_NAME'),
    'pool_size': int(os.getenv('MYSQL_POOL_SIZE')),
    'pool_reset_session': True,
    **db_config
}

# 커넥션 풀 생성
connection_pool = mysql.connector.pooling.MySQLConnectionPool(**pool_config)

# 커넥션 풀 사용 예시
def get_connection():
    return connection_pool.get_connection()

# 사용 예시
try:
    connection = get_connection()
    cursor = connection.cursor()
    
    sql_query = '''
        SELECT 1+1
    '''
    
    # 쿼리 실행
    cursor.execute(sql_query)
    results = cursor.fetchall()
    
    for row in results:
        print(row)
        
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()

(2,)


# 스키마 생성

In [6]:
import os
from mysql import connector

PASSWORD = 'kmh1234'

try:
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD
    ) as database: 
        print(f"Database object: {database}")
        delete_db = 'DROP DATABASE book_ratings;'       # 스키마 삭제 명령어
        create_db = 'CREATE DATABASE book_ratings;'     # 스키마 생성 명령어
        with database.cursor() as cursor:
            cursor.execute(create_db)

            # 현재 존재하는 스키마 목록 확인
            show_db = 'SHOW DATABASES;'
            cursor.execute(show_db)
            for db in cursor:
                print(db)
except connector.Error as e: 
    print(e)


Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x000002745FC7EAD0>
1007 (HY000): Can't create database 'book_ratings'; database exists


In [ ]:
import os
from mysql import connector

PASSWORD = 'kmh1234'

create_book_table = """
    CREATE TABLE books(
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(100),
        author VARCHAR(100),
        genre VARCHAR(100),
        release_year YEAR(4)
    );
"""

try:
    with connector.connect(
        host = "localhost",         # 데이터베이스 호스트 주소
        user="root",                # 사용자 이름
        password=PASSWORD,         # 비밀번호
        database = 'book_ratings'   # 사용할 데이터베이스 이름
    ) as database: 
        print(f"Database object: {database}")
        with database.cursor() as cursor:
            cursor.execute(create_book_table)   # 테이블 생성 쿼리 실행
            database.commit()                   # 변경 사항 저장(CREATE, INSERT 등은 commit 필요) 

            # table 확인
            describe_books = 'DESCRIBE books;'
            cursor.execute(describe_books)
            result = cursor.fetchall()  # 결과 모두 가져오기
                                        # 모든 테이블 정보를 다 가지고 오는데,  
                                        # pandas 데이터프레임으로 오는 것이 아니라, 
                                        # 리스트-튜플 형태로 저장
            for col in result:
                print(col)

except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x0000027400603750>
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('title', 'varchar(100)', 'YES', '', None, '')
('author', 'varchar(100)', 'YES', '', None, '')
('genre', 'varchar(100)', 'YES', '', None, '')
('release_year', 'year', 'YES', '', None, '')


In [ ]:
# INSERT SINGLE RECORD
insert_single_record = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
single_record = (
    "1", "Choose Yourself! Be Happy, Make Millions, Live the Dream", "James Altucher", "self-help", "2013"
    )

try: 
    # Connect to existing database
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD,
        database = "book_ratings"
    ) as existing_database:
        
        # Create cursor object
        with existing_database.cursor() as cursor:
            cursor.execute(insert_single_record, single_record)
            existing_database.commit()
        
except connector.Error as e: 
    print(e)

# INSERT MULTIPLE RECORDS
insert_multiple_records = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
multiple_records = [ # 리스트 형태로 튜플 데이터를 저장
    (
        "2", 
        "Skip the Line: The 10,000 Experiments Rule and Other Surprising Advice for Reaching Your Goals",
        "James Altucher",
        "self-help",
        "2021"        
    ),
    (
        "3",
        "The Power of No: Because One Little Word Can Bring Health, Abundance, and Happiness",
        "James Altucher",
        "self-help",
        "2014"
    ),
    (
        "4",
        "The 48 Laws of Power",
        "Robert Greene",
        "self-help",
        "1998"
    ),
    (
        "5",
        "Mastery",
        "Robert Greene",
        "self-help",
        "2012"
    ),
    (
        "6",
        "The Art of Seduction",
        "Robert Greene",
        "self-help",
        "2001"
    ),
]

try: 
    # Connect to existing database
    with connector.connect(
        host = "localhost",
        user = "root",
        password = PASSWORD,
        database = "book_ratings"
    ) as existing_database:
        
        # Create cursor object
        with existing_database.cursor() as cursor:
            cursor.executemany(insert_multiple_records, multiple_records)
            existing_database.commit()
            
except connector.Error as e: 
    print(e)

# 데이터 조회

In [27]:
try:
    with connector.connect(
        host = "localhost",  # 데이터베이스 호스트 주소     
        user = "root",       # 사용자 이름
        password = PASSWORD, # 비밀번호
        database = 'book_ratings'
    ) as database: 
        print(f"Database object: {database}")  # 연결된 데이터베이스 객체 출력

        # 조회
        select_query = 'SELECT author, release_year FROM books;'
        with database.cursor() as cursor:
            cursor.execute(select_query)

            # 결과 가져오기
            rows = cursor.fetchall()

            # 컬럼명 가져오기
            columns = [desc[0] for desc in cursor.description]

            # Pandas DataFrame으로 변환
            df = pd.DataFrame(rows, columns=columns)

            # 결과 출력
            print(df)


except connector.Error as e: 
    print(e)

Database object: <mysql.connector.connection_cext.CMySQLConnection object at 0x0000027400603ED0>
           author  release_year
0  James Altucher          2013
1  James Altucher          2021
2  James Altucher          2014
3   Robert Greene          1998
4   Robert Greene          2012
5   Robert Greene          2001


In [35]:
# 미션 : 어떤 형태로든 table을 조회해도 table의 고유한 칼럼 & 데이터프레임 

# .env 파일 로드
load_dotenv(dotenv_path='.env', override=True)
## override=True 옵션 : 기존에 로딩된 값 덮어씌움

# 변경된 환경변수 확인
print("DATABASE:", os.getenv("MYSQL_DATABASE"))

# 데이터베이스 연결 설정
db_config = {
    'host': os.getenv('MYSQL_HOST'),
    'port': int(os.getenv('MYSQL_PORT')),
    'user': os.getenv('MYSQL_USER'),
    'password': os.getenv('MYSQL_PASSWORD'),
    'database': os.getenv('MYSQL_DATABASE'),
    'charset': os.getenv('MYSQL_CHARSET')
}

# 커넥션 풀 설정
pool_config = {
    'pool_name': os.getenv('MYSQL_POOL_NAME'),
    'pool_size': int(os.getenv('MYSQL_POOL_SIZE')),
    'pool_reset_session': True,
    **db_config
}

# 커넥션 풀 생성
connection_pool = mysql.connector.pooling.MySQLConnectionPool(**pool_config)

# 커넥션 풀 사용 예시
def get_connection():
    return connection_pool.get_connection()

# 사용 예시
try:
    connection = get_connection()
    cursor = connection.cursor()
    
     # 조회
    select_query = 'SELECT author, release_year FROM books;'
    cursor.execute(select_query)

    # 결과 가져오기
    rows = cursor.fetchall()

    # 컬럼명 가져오기
    columns = [desc[0] for desc in cursor.description]

    # Pandas DataFrame으로 변환
    df = pd.DataFrame(rows, columns=columns)

    # 결과 출력
    print(df)

        
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()

DATABASE: book_ratings
           author  release_year
0  James Altucher          2013
1  James Altucher          2021
2  James Altucher          2014
3   Robert Greene          1998
4   Robert Greene          2012
5   Robert Greene          2001


In [ ]:
from dotenv import load_dotenv
import os
import mysql.connector
from mysql.connector import pooling

# .env 파일 로드
load_dotenv(dotenv_path='.env', override=True)

# 데이터베이스 연결 설정
db_config = {
    'host': os.getenv('MYSQL_HOST'),
    'port': int(os.getenv('MYSQL_PORT')),
    'user': os.getenv('MYSQL_USER'),
    'password': os.getenv('MYSQL_PASSWORD'),
    'database': os.getenv('MYSQL_DATABASE'),
    'charset': os.getenv('MYSQL_CHARSET')
}

# 커넥션 풀 설정
pool_config = {
    'pool_name': os.getenv('MYSQL_POOL_NAME'),
    'pool_size': int(os.getenv('MYSQL_POOL_SIZE')),
    'pool_reset_session': True,
    **db_config
}

# 커넥션 풀 생성
connection_pool = mysql.connector.pooling.MySQLConnectionPool(**pool_config)

# 커넥션 풀 사용 예시
def get_connection():
    return connection_pool.get_connection()

import pandas as pd

def fetch_table_as_dataframe(table_name: str) -> pd.DataFrame:
    """
    커넥션 풀에서 연결을 받아 지정된 테이블을 조회하고 DataFrame으로 반환.

    Parameters:
        table_name (str): 조회할 테이블 이름

    Returns:
        pd.DataFrame: 테이블 내용을 담은 데이터프레임
    """
    try:
        # 커넥션 풀에서 연결 가져오기
        connection = connection_pool.get_connection()
        cursor = connection.cursor()
        query = f"SELECT * FROM {table_name};"
        cursor.execute(query)

        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]

        return pd.DataFrame(rows, columns=columns)

    except mysql.connector.Error as err:
        print(f"❗ MySQL 에러 발생: {err}")
        return pd.DataFrame()

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals() and connection.is_connected():
            connection.close()

df = fetch_table_as_dataframe("books")
print(df.head())


   id                                              title          author  \
0   1  Choose Yourself! Be Happy, Make Millions, Live...  James Altucher   
1   2  Skip the Line: The 10,000 Experiments Rule and...  James Altucher   
2   3  The Power of No: Because One Little Word Can B...  James Altucher   
3   4                               The 48 Laws of Power   Robert Greene   
4   5                                            Mastery   Robert Greene   

       genre  release_year  
0  self-help          2013  
1  self-help          2021  
2  self-help          2014  
3  self-help          1998  
4  self-help          2012  


In [ ]:
# [1단계] 필요 라이브러리 불러오기
from dotenv import load_dotenv
import os
import mysql.connector
from mysql.connector import pooling
import pandas as pd

# [2단계] .env 파일 로드
load_dotenv(dotenv_path='.env', override=True)

# [3단계] DB 설정값 환경변수에서 불러오기
db_config = {
    'host': os.getenv('MYSQL_HOST'),
    'port': int(os.getenv('MYSQL_PORT')),
    'user': os.getenv('MYSQL_USER'),
    'password': os.getenv('MYSQL_PASSWORD'),
    'database': os.getenv('MYSQL_DATABASE'),
    'charset': os.getenv('MYSQL_CHARSET')
}

# [4단계] 커넥션 풀 생성 (애플리케이션 전체에서 1회만 실행)
connection_pool = pooling.MySQLConnectionPool(
    pool_name=os.getenv('MYSQL_POOL_NAME'),           # 예: mypool
    pool_size=int(os.getenv('MYSQL_POOL_SIZE')),      # 예: 5
    pool_reset_session=True,                          # 세션 초기화 여부
    **db_config
)

# ──────────────────────────────────────────────────────────────
# [5단계-1] 단순 조회용 (커넥션을 함수 내부에서 꺼냄 - 추천: 단건 조회용)
def fetch_table_as_dataframe(table_name: str) -> pd.DataFrame:
    """
    지정된 테이블을 커넥션 풀에서 연결을 꺼내 조회하여 DataFrame으로 반환 (단건 조회 전용).
    """
    try:
        connection = connection_pool.get_connection()
        cursor = connection.cursor()
        query = f"SELECT * FROM {table_name};"
        cursor.execute(query)

        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(rows, columns=columns)

    except mysql.connector.Error as err:
        print(f"❗ MySQL 에러 발생: {err}")
        return pd.DataFrame()

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals() and connection.is_connected():
            connection.close()

# ──────────────────────────────────────────────────────────────
# [5단계-2] 다중 쿼리/재사용용 (커넥션을 함수 외부에서 주입)
def fetch_table_with_connection(conn, table_name: str) -> pd.DataFrame:
    """
    주어진 커넥션을 사용해 테이블을 조회하여 DataFrame으로 반환 (재사용/트랜잭션용).
    """
    try:
        cursor = conn.cursor()
        query = f"SELECT * FROM {table_name};"
        cursor.execute(query)

        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        return pd.DataFrame(rows, columns=columns)

    except mysql.connector.Error as err:
        print(f"❗ MySQL 에러 발생: {err}")
        return pd.DataFrame()

    finally:
        if 'cursor' in locals():
            cursor.close()

# ──────────────────────────────────────────────────────────────
# [6단계] 함수 사용 예시

# ✅ 방법 1: 단건 조회용 (내부에서 커넥션 꺼냄)
df1 = fetch_table_as_dataframe("books")
print("📘 books 테이블 결과:")
print(df1.head())

# ✅ 방법 2: 외부 커넥션 주입 방식 (복수 테이블 조회 시 유리)
conn = connection_pool.get_connection()
df2 = fetch_table_with_connection(conn, "books")
conn.close()  # 마지막에 커넥션 반환

print("\n📘 books 테이블 결과 2:")
print(df2.head())
